# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [1]:

# imports

import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd

import plotly.express as px

import module as mod     # functions are include in module.py
#%pip install cufflinks

import cufflinks as cf
cf.go_offline()


import matplotlib.pyplot as plt   

import seaborn as sns

import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [3]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [4]:
#deaths.head()

In [5]:
deaths['Causa de muerte'].unique()


array(['001-102  I-XXII.Todas las causas',
       '001-008  I.Enfermedades infecciosas y parasitarias',
       '001  Enfermedades infecciosas intestinales',
       '002  Tuberculosis y sus efectos tardíos',
       '003  Enfermedad meningocócica', '004  Septicemia',
       '005  Hepatitis vírica', '006  SIDA',
       '007  VIH+ (portador, evidencias de laboratorio del VIH, ...)',
       '008  Resto de enfermedades infecciosas y parasitarias y sus efectos tardíos',
       '009-041  II.Tumores',
       '009  Tumor maligno del labio, de la cavidad bucal y de la faringe',
       '010  Tumor maligno del esófago',
       '011  Tumor maligno del estómago', '012  Tumor maligno del colon',
       '013  Tumor maligno del recto, de la porción rectosigmoide y del ano',
       '014  Tumor maligno del hígado y vías biliares intrahepáticas',
       '015  Tumor maligno del páncreas',
       '016  Otros tumores malignos digestivos',
       '017  Tumor maligno de la laringe',
       '018  Tumor maligno d

In [6]:
deaths.describe()['Total']

count    301158.000000
mean        553.335937
std        5904.622883
min           0.000000
25%           1.000000
50%          12.000000
75%         100.000000
max      427721.000000
Name: Total, dtype: float64

In [7]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [8]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name',]].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table.head()

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica


In [9]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())

# different causes Single and Multiple

pd.set_option('display.max_rows', None)

causes_table_type = deaths[['cause_code', 'cause_name','cause_group']].drop_duplicates('cause_name').sort_values(by='cause_code').reset_index(drop=True)

causes_table_type_single  = causes_table_type.loc[causes_table_type.cause_group == 'Single cause'].reset_index(drop=True)
causes_table_type_single.head()


,cause_code,cause_name,cause_group
0,001,Enfermedades infecciosas intestinales,Single cause
1,002,Tuberculosis y sus efectos tardíos,Single cause
2,003,Enfermedad meningocócica,Single cause
3,004,Septicemia,Single cause
4,005,Hepatitis vírica,Single cause


In [10]:
causes_table_type_multiple  = causes_table_type.loc[causes_table_type ['cause_group'] == 'Multiple causes'].reset_index(drop=True)
causes_table_type_multiple


,cause_code,cause_name,cause_group
0,001-008,I.Enfermedades infecciosas y parasitarias,Multiple causes
1,001-102,I-XXII.Todas las causas,Multiple causes
2,009-041,II.Tumores,Multiple causes
3,042-043,III.Enfermedades de la sangre y de los órganos...,Multiple causes
4,044-045,"IV.Enfermedades endocrinas, nutricionales y me...",Multiple causes
5,046-049,V.Trastornos mentales y del comportamiento,Multiple causes
6,050-052,VI-VIII.Enfermedades del sistema nervioso y de...,Multiple causes
7,053-061,IX.Enfermedades del sistema circulatorio,Multiple causes
8,062-067,X.Enfermedades del sistema respiratorio,Multiple causes
9,068-072,XI.Enfermedades del sistema digestivo,Multiple causes


**TABLA TODAS ENFERMEDADES**

In [11]:
#Todas las enfermedades
causes_todas_enfermedades = deaths[['cause_code', 'cause_name','cause_group']].drop_duplicates('cause_name').sort_values(by='cause_code').reset_index(drop=True)

causes_table_type_single  = causes_table_type.loc[causes_table_type.cause_group == 'Single cause'].reset_index(drop=True)
causes_table_type_single.head()


,cause_code,cause_name,cause_group
0,001,Enfermedades infecciosas intestinales,Single cause
1,002,Tuberculosis y sus efectos tardíos,Single cause
2,003,Enfermedad meningocócica,Single cause
3,004,Septicemia,Single cause
4,005,Hepatitis vírica,Single cause


In [12]:
#Analize 0
ceros=deaths.loc[deaths.Total == 0]
pd.set_option('display.max_rows', None)
ceros.loc[ceros.cause_code== '046-049'].head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
128739,046-049 V.Trastornos mentales y del comportam...,Total,Menos de 1 año,2018,0,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128740,046-049 V.Trastornos mentales y del comportam...,Total,Menos de 1 año,2017,0,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128741,046-049 V.Trastornos mentales y del comportam...,Total,Menos de 1 año,2016,0,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128742,046-049 V.Trastornos mentales y del comportam...,Total,Menos de 1 año,2015,0,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128743,046-049 V.Trastornos mentales y del comportam...,Total,Menos de 1 año,2014,0,046-049,Multiple causes,V.Trastornos mentales y del comportamiento


In [13]:
deaths.loc[deaths.cause_code== '046-049'].head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
128700,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2018,22376,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128701,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2017,21722,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128702,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2016,20980,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128703,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2015,21333,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128704,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2014,18706,046-049,Multiple causes,V.Trastornos mentales y del comportamiento


In [14]:
#DataFrame Enfermedades mentales

deaths_mentales = deaths.loc[(deaths['cause_code'] >= '046') & (deaths['cause_code'] <= '049')]
deaths_mentales.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
128700,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2018,22376,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128701,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2017,21722,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128702,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2016,20980,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128703,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2015,21333,046-049,Multiple causes,V.Trastornos mentales y del comportamiento
128704,046-049 V.Trastornos mentales y del comportam...,Total,Todas las edades,2014,18706,046-049,Multiple causes,V.Trastornos mentales y del comportamiento


## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

## ...and finally, show me some insights with Plotly!!!

In [15]:
# Cufflinks histogram

dataset_todos_trastornos_mentales = mod.row_filter(deaths_mentales, 'Sexo', ['Total'])
dataset_todos_trastornos_mentales= mod.row_filter(dataset_todos_trastornos_mentales, 'Edad', ['Todas las edades'])
dataset_todos_trastornos_mentales = mod.row_filter(dataset_todos_trastornos_mentales, 'cause_group', ['Multiple causes'])
dataset_todos_trastornos_mentales.head()

dataset_column=dataset_todos_trastornos_mentales.Total

dataset_column.iplot(kind='hist',
                     title='Trastornos Mentales',
                     yTitle='Total',
                     xTitle='Num muertes')


In [16]:
# Cufflinks bar plot

dataset_bar=dataset_todos_trastornos_mentales


dataset_bar.iplot(kind='bar',
                  x='Periodo',
                  y='Total',
                  xTitle='Años',
                  yTitle='Total muertes',
                  title='Muertes anuales por trastornos mentales ')


In [17]:
dataset_todos_enfermedades = mod.row_filter(deaths, 'Sexo', ['Total'])
dataset_todos_enfermedades = mod.row_filter(dataset_todos_enfermedades, 'cause_group', ['Multiple causes'])

group = ['cause_code','cause_name','Edad']
dataset_total = mod.groupby_sum(dataset_todos_enfermedades, group)

In [20]:
dataset_todos_enfermedades = mod.row_filter(deaths, 'Sexo', ['Total'])
dataset_todos_enfermedades = mod.row_filter(dataset_todos_enfermedades, 'cause_group', ['Multiple causes'])


group = ['cause_code','cause_name','Edad']
dataset_total = mod.groupby_sum(dataset_todos_enfermedades, group)

#eliminar otras causas
dataset_filtrado = dataset_total[dataset_total['cause_code'] != '090-102']

#grouping all non-mental illnesses together
resto_enfermedades= [
 '053-061',
 '009-041',
 '062-067',
 '068-072',
 '050-052',
 '044-045',
 '086-089',
 '077-080',
 '001-008',
 '074-076',
 '042-043',
 '083-085']
dataset_filtrado.cause_code=dataset_filtrado.cause_code.replace(resto_enfermedades, '001')

dataset_filtrado.loc[dataset_filtrado['cause_code'] == '001', 'cause_name'] = 'Resto de enfermedades'

#eliminate all causes and all ages
dataset_filtrado= dataset_filtrado.drop(dataset_filtrado.loc[dataset_filtrado['cause_code'] == '001-102'].index)
dataset_filtrado= dataset_filtrado.drop(dataset_filtrado.loc[dataset_filtrado['Edad'] == 'Todas las edades'].index)


replace_dict={'De 80 a 84 años  ':'De 80 a 90',
 'De 85 a 89 años  ':'De 80 a 90',
 'De 75 a 79 años  ':'De 70 a 80',
 'De 90 a 94 años  ':'De 90 en adelante', 
 'De 70 a 74 años  ':'De 70 a 80',
 'De 65 a 69 años':'De 60 a 70',
 'De 60 a 64 años':'De 60 a 70',
 'De 55 a 59 años':'De 50 a 60',
 '95 y más años':'De 90 en adelante' ,
 'De 50 a 54 años':'De 50 a 60',
 'De 45 a 49 años':'De 40 a 50',
 'De 40 a 44 años':'De 40 a 50',
 'De 35 a 39 años':'De 30 a 40',
 'Menos de 1 año':'De 0 a 10' ,
 'De 30 a 34 años':'De 30 a 40',
 'De 25 a 29 años':'De 20 a 30',
 'De 20 a 24 años':'De 20 a 30',
 'De 15 a 19 años  ':'De 10 a 20',
 'De 5 a 9 años':'De 0 a 10' ,
 'De 10 a 14 años  ':'De 10 a 20',
 'De 1 a 4 años':'De 0 a 10'} 
dataset_filtrado['Edad'] = dataset_filtrado['Edad'].replace(replace_dict)

#crear pivot table por resto de enfermedades y enfermedades mentales

dataset_filtrados = mod.pivot_table(dataset_filtrado, 'cause_code', 'Edad')
dataset_filtrados .rename(columns={'001': 'Resto de enfermedades', '046-049': 'Enfermedades mentales'}, inplace=True)


#crear una tabla con el porcentaje de muertes por enfermedades mentales frente a las muertes por enfermedades total  
dataset_filtrado_porcentaje=dataset_filtrados
dataset_filtrado_porcentaje['Total'] = dataset_filtrado_porcentaje['Enfermedades mentales'] + dataset_filtrado_porcentaje['Resto de enfermedades']

dataset_filtrado_porcentaje['Porcentaje'] = round((dataset_filtrado_porcentaje['Enfermedades mentales'] / dataset_filtrado_porcentaje['Total']) * 100,2)

dataset_filtrado = mod.pivot_table(dataset_filtrado, 'cause_code', 'Edad')
dataset_filtrado .rename(columns={'001': 'Resto de enfermedades', '046-049': 'Enfermedades mentales'}, inplace=True)




In [21]:

dataset_bar=dataset_filtrado

dataset_bar.iplot(kind='bar',
                  x='Edad',
                  xTitle='Edad',
                  yTitle='Total muertes',
                  title='Muertes por enfermedades mentales vs resto de enfermedades')


In [22]:
group = ['cause_code','cause_name','Periodo']
dataset_total = mod.groupby_sum(dataset_todos_enfermedades, group)
dataset_filtrado = dataset_total[dataset_total['cause_code'] != '090-102']
dataset_bar=dataset_filtrado

dataset_filtrado.cause_code=dataset_filtrado.cause_code.replace(resto_enfermedades, '001')

dataset_filtrado.loc[dataset_filtrado['cause_code'] == '001', 'cause_name'] = 'Resto de enfermedades'

dataset_filtrado= dataset_filtrado.drop(dataset_filtrado.loc[dataset_filtrado['cause_code'] == '001-102'].index)
dataset_filtrado = mod.pivot_table(dataset_filtrado, 'cause_code', 'Periodo')
dataset_filtrado .rename(columns={'001': 'Resto de enfermedades', '046-049': 'Enfermedades mentales'}, inplace=True)


In [23]:
dataset_bar=dataset_filtrado
dataset_bar.iplot(kind='bar',
                  x='Periodo',
                  xTitle='Años',
                  yTitle='Total muertes',
                  title='Muertes por enfermedades mentales vs resto de enfermedades')

In [24]:
dataset_line = dataset_filtrado_porcentaje.sort_values(by='Edad')

scatter_plot = dataset_line.iplot(kind='scatter',
                                  x='Edad',
                                  y='Porcentaje',
                                  xTitle='Edad',
                                  yTitle='Porcentaje',
                                  mode='lines',
                                  colors=['green'],
                                  title='Porcentaje de muertes por enfermedad mental sobre las muertes por resto de enfermedades')

scatter_plot

In [25]:

dataset_bar = dataset_filtrados

# Create the bar plot
bar_plot = dataset_bar.iplot(kind='bar', x='Edad', y=['Enfermedades mentales', 'Resto de enfermedades'], 
                             title='Muertes por enfermedades mentales vs resto de enfermedades',
                             xTitle='Edad', yTitle='Total muertes',
                             barmode='group', asFigure=True)

# Sort dataset_filtrado_porcentaje by 'Edad'
dataset_line = dataset_filtrado_porcentaje.sort_values(by='Edad')

# Create the scatter trace
scatter_trace = {
    'x': dataset_line['Edad'],
    'y': dataset_line['Porcentaje'],
    'line': {'color': 'green'},
    'name': 'Porcentaje (%)',
    'type': 'scatter',
    'yaxis': 'y2'
}

combined = bar_plot.add_traces(scatter_trace)
combined['layout']['yaxis2'] = dict(title='Porcentaje (%)', overlaying='y', side='right', range=[0, 50])
combined

In [26]:
dataset_todos_enfermedades_sexo = mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
dataset_todos_enfermedades_sexo = mod.row_filter(dataset_todos_enfermedades_sexo, 'cause_group', ['Multiple causes'])
dataset_todos_enfermedades_sexo ['Edad'] = dataset_todos_enfermedades_sexo ['Edad'].replace(replace_dict)
dataset_todos_enfermedades_sexo = dataset_todos_enfermedades_sexo[dataset_todos_enfermedades_sexo['Edad'] != 'Todas las edades']
group = ['Edad','Sexo']
dataset_todos_enfermedades_sexo = mod.groupby_sum(dataset_todos_enfermedades_sexo, group)
dataset_todos_enfermedades_sexo = mod.pivot_table(dataset_todos_enfermedades_sexo, 'Sexo', 'Edad')

dataset_bar=dataset_todos_enfermedades_sexo

dataset_bar.iplot(kind='bar',
                  x='Edad',
                  xTitle='Edad',
                  yTitle='Total muertes',
                  title='Muertes por enfermedades mentales Hombres vs Mujeres')

In [27]:
dataset_todos_enfermedades_sexo_v1 = mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
dataset_todos_enfermedades_sexo_v1 = mod.row_filter(dataset_todos_enfermedades_sexo_v1, 'cause_group', ['Multiple causes'])
dataset_todos_enfermedades_sexo_v1 = mod.row_filter(dataset_todos_enfermedades_sexo_v1, 'Edad', ['Todas las edades'])
group = ['Periodo','Sexo']
dataset_todos_enfermedades_sexo_v1= mod.groupby_sum(dataset_todos_enfermedades_sexo_v1, group)
dataset_todos_enfermedades_sexo_v1 = mod.pivot_table(dataset_todos_enfermedades_sexo_v1, 'Sexo', 'Periodo')



dataset_bar=dataset_todos_enfermedades_sexo_v1

dataset_bar.iplot(kind='bar',
                  x='Periodo',
                  xTitle='Año',
                  yTitle='Total muertes',
                  title='Muertes por enfermedades mentales Hombres vs Mujeres')

In [28]:
dataset_todos_enfermedades_mentales = mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
dataset_todos_enfermedades_mentales = mod.row_filter(dataset_todos_enfermedades_mentales, 'Edad', ['Todas las edades'])
dataset_todos_enfermedades_mentales = mod.row_filter(dataset_todos_enfermedades_mentales, 'cause_group', ['Single cause'])
group = ['Causa de muerte', 'Sexo']
dataset_todos_enfermedades_mentales= mod.groupby_sum(dataset_todos_enfermedades_mentales, group)
dataset_todos_enfermedades_mentales = mod.pivot_table(dataset_todos_enfermedades_mentales, 'Sexo', 'Causa de muerte')

dataset_todos_enfermedades_mentales = dataset_todos_enfermedades_mentales.sort_values(by='Causa de muerte', ascending=True)



dataset_bar=dataset_todos_enfermedades_mentales
dataset_bar.iplot(kind='barh', x='Causa de muerte',
                  title='Muertes por las distintas enfermedades mentales',
                  xTitle='Total muertes', yTitle='Causa de muerte')

             

In [29]:
dataset_todos_enfermedades_sexo_v2= mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
dataset_todos_enfermedades_sexo_v2 = mod.row_filter(dataset_todos_enfermedades_sexo_v2, 'cause_code', ['046'])
dataset_todos_enfermedades_sexo_v2 ['Edad'] = dataset_todos_enfermedades_sexo_v2 ['Edad'].replace(replace_dict)
dataset_todos_enfermedades_sexo_v2 = dataset_todos_enfermedades_sexo_v2[dataset_todos_enfermedades_sexo_v2['Edad'] != 'Todas las edades']
group = ['Edad','Sexo']
dataset_todos_enfermedades_sexo_v2 = mod.groupby_sum(dataset_todos_enfermedades_sexo_v2, group)
dataset_todos_enfermedades_sexo_v2 = mod.pivot_table(dataset_todos_enfermedades_sexo_v2, 'Sexo', 'Edad')

dataset_bar=dataset_todos_enfermedades_sexo_v2


dataset_bar.iplot(kind='bar',
                  x='Edad',
                  xTitle='Edad',
                  yTitle='Total muertes',
                  title='Muertes por trastornos mentales organicos, senil, presenil Hombres vs Mujeres')

In [30]:
dataset_todos_enfermedades_sexo_v2= mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
dataset_todos_enfermedades_sexo_v2 = mod.row_filter(dataset_todos_enfermedades_sexo_v2, 'cause_code', ['046'])
dataset_todos_enfermedades_sexo_v2 ['Edad'] = dataset_todos_enfermedades_sexo_v2 ['Edad'].replace(replace_dict)
dataset_todos_enfermedades_sexo_v2 = dataset_todos_enfermedades_sexo_v2[dataset_todos_enfermedades_sexo_v2['Edad'] != 'Todas las edades']

ultimos_tres_rangos = dataset_todos_enfermedades_sexo_v2['Edad'].unique()[:3]

dataset_ultimos_tres_rangos = dataset_todos_enfermedades_sexo_v2[dataset_todos_enfermedades_sexo_v2['Edad'].isin(ultimos_tres_rangos)]

datos_agrupados = dataset_ultimos_tres_rangos.groupby(['Periodo', 'Edad'])['Total'].sum().unstack()

fig = go.Figure()

for edad in datos_agrupados.columns:
    fig.add_trace(go.Bar(x=datos_agrupados.index, y=datos_agrupados[edad], name=edad))

fig.update_layout(barmode='group', xaxis_title='Período', yaxis_title='Total de Muertes', title='Muertes por trastornos organicos, senil y presenil para los Tres Últimos Rangos de Edad')

fig.show()


In [32]:
dataset_todos_enfermedades_sexo_v2= mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
dataset_todos_enfermedades_sexo_v2 = mod.row_filter(dataset_todos_enfermedades_sexo_v2, 'cause_code', ['047'])
dataset_todos_enfermedades_sexo_v2 ['Edad'] = dataset_todos_enfermedades_sexo_v2 ['Edad'].replace(replace_dict)
dataset_todos_enfermedades_sexo_v2 = dataset_todos_enfermedades_sexo_v2[dataset_todos_enfermedades_sexo_v2['Edad'] != 'Todas las edades']
group = ['Edad','Sexo']
dataset_todos_enfermedades_sexo_v2 = mod.groupby_sum(dataset_todos_enfermedades_sexo_v2, group)
dataset_todos_enfermedades_sexo_v2 = mod.pivot_table(dataset_todos_enfermedades_sexo_v2, 'Sexo', 'Edad')

dataset_bar=dataset_todos_enfermedades_sexo_v2


dataset_bar.iplot(kind='bar',
                  x='Edad',
                  xTitle='Edad',
                  yTitle='Total muertes',
                  title='Muertes por trastornos mentales por uso de alcohol Hombres vs Mujeres')

In [33]:
dataset_todos_enfermedades_sexo_v3= mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
dataset_todos_enfermedades_sexo_v3 = mod.row_filter(dataset_todos_enfermedades_sexo_v3, 'cause_code', ['048'])
dataset_todos_enfermedades_sexo_v3 ['Edad'] = dataset_todos_enfermedades_sexo_v3 ['Edad'].replace(replace_dict)
dataset_todos_enfermedades_sexo_v3 = dataset_todos_enfermedades_sexo_v3 [dataset_todos_enfermedades_sexo_v3['Edad'] != 'Todas las edades']
group = ['Edad','Sexo']
dataset_todos_enfermedades_sexo_v3 = mod.groupby_sum(dataset_todos_enfermedades_sexo_v3, group)
dataset_todos_enfermedades_sexo_v3 = mod.pivot_table(dataset_todos_enfermedades_sexo_v3, 'Sexo', 'Edad')


dataset_bar=dataset_todos_enfermedades_sexo_v3


dataset_bar.iplot(kind='bar',
                  x='Edad',
                  xTitle='Edad',
                  yTitle='Total muertes',
                  title='Muertes por trastornos mentales por uso de drogas Hombres vs Mujeres')

In [35]:
dataset_todos_enfermedades_mentales = mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])

dataset_todos_enfermedades_mentales = mod.row_filter(dataset_todos_enfermedades_mentales, 'cause_group', ['Single cause'])
dataset_todos_enfermedades_mentales= dataset_todos_enfermedades_mentales.drop(dataset_todos_enfermedades_mentales.loc[dataset_todos_enfermedades_mentales['Edad'] == 'Todas las edades'].index)
dataset_todos_enfermedades_mentales ['Edad'] = dataset_todos_enfermedades_mentales ['Edad'].replace(replace_dict)
group = ['Causa de muerte', 'Sexo','Edad']
dataset_todos_enfermedades_mentales= mod.groupby_sum(dataset_todos_enfermedades_mentales, group)
dataset_todos_enfermedades_mentales = mod.row_filter(dataset_todos_enfermedades_mentales, 'Edad', ['De 0 a 10', 'De 10 a 20', 'De 20 a 30'])

dataset_todos_enfermedades_mentales = mod.pivot_table(dataset_todos_enfermedades_mentales, 'Causa de muerte','Edad')



dataset_bar=dataset_todos_enfermedades_mentales
dataset_bar.iplot(kind='barh', x='Edad',
                  title='Muertes por las distintas enfermedades mentales para los tres primeros grupos de edad',
                  xTitle='Total muertes', yTitle='Causa de muerte')


In [37]:
dataset_todos_enfermedades_mentales = mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])

dataset_todos_enfermedades_mentales = mod.row_filter(dataset_todos_enfermedades_mentales, 'cause_group', ['Single cause'])
dataset_todos_enfermedades_mentales= dataset_todos_enfermedades_mentales.drop(dataset_todos_enfermedades_mentales.loc[dataset_todos_enfermedades_mentales['Edad'] == 'Todas las edades'].index)
dataset_todos_enfermedades_mentales ['Edad'] = dataset_todos_enfermedades_mentales ['Edad'].replace(replace_dict)
group = ['Causa de muerte', 'Sexo','Edad']
dataset_todos_enfermedades_mentales= mod.groupby_sum(dataset_todos_enfermedades_mentales, group)
dataset_todos_enfermedades_mentales = mod.row_filter(dataset_todos_enfermedades_mentales, 'Edad', ['De 0 a 10'])

dataset_todos_enfermedades_mentales = mod.pivot_table(dataset_todos_enfermedades_mentales, 'Causa de muerte','Edad')



dataset_bar=dataset_todos_enfermedades_mentales
dataset_bar.iplot(kind='barh', x='Edad',
                  title='Muertes por las distintas enfermedades mentales de 0 a 10 años',
                  xTitle='Total muertes', yTitle='Causa de muerte')

In [40]:
dataset_todos_enfermedades_sexo_v2= mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
#dataset_todos_enfermedades_sexo_v2 = mod.row_filter(dataset_todos_enfermedades_sexo_v2, 'cause_code', ['046'])
dataset_todos_enfermedades_sexo_v2 ['Edad'] = dataset_todos_enfermedades_sexo_v2 ['Edad'].replace(replace_dict)
dataset_todos_enfermedades_sexo_v2 = dataset_todos_enfermedades_sexo_v2[dataset_todos_enfermedades_sexo_v2['Edad'] != 'Todaslas edades']
dataset_todos_enfermedades_sexo_v2 = mod.row_filter(dataset_todos_enfermedades_sexo_v2, 'Edad',['De 0 a 10', 'De 10 a 20', 'De 20 a 30'])

group = ['Edad','Sexo']
dataset_todos_enfermedades_sexo_v2 = mod.groupby_sum(dataset_todos_enfermedades_sexo_v2, group)
dataset_todos_enfermedades_sexo_v2 = mod.pivot_table(dataset_todos_enfermedades_sexo_v2, 'Sexo', 'Edad')
dataset_bar=dataset_todos_enfermedades_sexo_v2 

dataset_bar.iplot(kind='bar',
                  x='Edad',
                  xTitle='Edad',
                  yTitle='Total muertes',
                  title='Muertes por trastornos mentales para los tres primeros grupos de edad. Hombres vs Mujeres')

In [42]:
dataset_todos_enfermedades_sexo_v2= mod.row_filter(deaths_mentales, 'Sexo', ['Mujeres','Hombres'])
#dataset_todos_enfermedades_sexo_v2 = mod.row_filter(dataset_todos_enfermedades_sexo_v2, 'cause_code', ['046'])
dataset_todos_enfermedades_sexo_v2 ['Edad'] = dataset_todos_enfermedades_sexo_v2 ['Edad'].replace(replace_dict)
dataset_todos_enfermedades_sexo_v2 = dataset_todos_enfermedades_sexo_v2[dataset_todos_enfermedades_sexo_v2['Edad'] != 'Todas las edades']

ultimos_tres_rangos = dataset_todos_enfermedades_sexo_v2['Edad'].unique()[-3:]

dataset_ultimos_tres_rangos = dataset_todos_enfermedades_sexo_v2[dataset_todos_enfermedades_sexo_v2['Edad'].isin(ultimos_tres_rangos)]

datos_agrupados = dataset_ultimos_tres_rangos.groupby(['Periodo', 'Edad'])['Total'].sum().unstack()

fig = go.Figure()

for edad in datos_agrupados.columns:
    fig.add_trace(go.Bar(x=datos_agrupados.index, y=datos_agrupados[edad], name=edad))

fig.update_layout(barmode='group', xaxis_title='Período', yaxis_title='Total de Muertes', title='Evolución muertes por Período para los Tres Primeros Rangos de Edad')

fig.show()

In [43]:
# Cufflinks line plot
dataset_line=dataset_todos_trastornos_mentales.sort_values(by='Periodo')
dataset_line.iplot(kind='line',
                   x='Periodo',
                   y='Total',
                   xTitle='Años',
                   yTitle='Total muertes',
                   title='Evolución muertes por enfermedades mentales')


In [44]:
dataset_single_trastornos_mentales = mod.row_filter(deaths_mentales, 'Sexo', ['Total'])
dataset_single_trastornos_mentales= mod.row_filter(dataset_single_trastornos_mentales, 'Edad', ['Todas las edades'])
dataset_single_trastornos_mentales = mod.row_filter(dataset_single_trastornos_mentales, 'cause_group', ['Single cause'])
group = ['Causa de muerte','Periodo']
dataset_single_trastornos_mentales = mod.groupby_sum(dataset_single_trastornos_mentales, group)
dataset_single_trastornos_mentales = mod.pivot_table(dataset_single_trastornos_mentales, 'Causa de muerte','Periodo')

dataset_line=dataset_single_trastornos_mentales.sort_values(by='Periodo')
dataset_line.iplot(kind='line',
                   x='Periodo',
                   xTitle='Años',
                   yTitle='Total muertes',
                   title='Evolución muertes por las distintas enfermedades mentales')


In [45]:


df = dataset_single_trastornos_mentales.sort_values(by='Periodo')

df.set_index('Periodo', inplace=True)


ultimo_año = df.tail(1)


bar_chart = ultimo_año.iplot(kind='barh', title='Muertes por las distintas enfermedades mentales')


area_chart = df.iloc[:-1].iplot(kind='area', fill=True, theme='white', title='Evolución muertes por las distintas enfermedades mentales')


fig = subplots.make_subplots(rows=1, cols=2)


fig.add_trace(bar_chart.data[0], row=1, col=1)
fig.add_trace(area_chart.data[0], row=1, col=2)

fig.update_layout(title='Muertes por las distintas enfermedades mentales')

pyo.plot(fig, filename='comparacion_ultimo_anio.html')

NameError: name 'subplots' is not defined

 # Conclusiones

1. Las muertes por trastornos mentales han aumentado en los ultimos años 

2. Aunque el % de muertes por trastornos mentales sobre las muertes por resto de enfermedades es muy bajo, ha crecido a lo largo de los años hasta superar el 5%

3. A partir de los 70 años las muertes por trastornos mentales de mujeres es superior a la de los hombres.

4. Conforme avanza la edad las muertes por trastorno mental son superiores. 

5. Por años, en general las muertes de mujeres por trastornos mentales son superiores. 

6. La causa con mayor numero de muertes de trastornos mentales son por trastornos organicos, senil y presenil. Sobre todo en las mujeres. Además conforme pasa los años son mayores las muertes por este tipo de trastorno a partir de los 80 años, y de los 70  a los 80 se mantiene más estable. 


7. Las muertes por uso de alcohol es mayor en hombres que en mujeres, y en edades entre 40-80 años. Han caido a lo largo de los años.

8. Las muertes por uso de droga es mayor en hombres que en mujeres, y en edades entre 20-50 años.

9. Entre los 0-20 años las principales muertes por trastornos mentales son por otros trastornos mentales y de comportamiento. En cambio, de los 20- a los 30 son por uso de drogas. 

10. Por sexo, las muertes de hombres es más que el doble que las mujeres de los 20 a los 30 y ligeramente superiores de los 10 a los 20 años. De 0 a 10 son similares y poco cuantiosas. 






In [ ]:
1. 